In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re

In [2]:
xls = pd.ExcelFile("Product Matching Dataset.xlsx")

In [3]:
master_df = pd.read_excel(xls, "Master File")
dataset_df = pd.read_excel(xls, "Dataset")

In [4]:
dataset_df.head()

sku marketplace_product_name_ar       seller_item_name  price
0  1322    استوهالت 40 مجم 14 كبسول  ESTOHALT 40 MG 14 CAP   56.5
1  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5
2  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5
3  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5
4  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5

In [5]:
master_df.head()

sku                    product_name                product_name_ar  price
0   279          ANAFRONIL 75 MG 20 TAB  انافرونيل 75 مجم اس ار 20 قرص   75.0
1  2282         LOPRECOUGH SYRUP 100 ML           لوبريكاف شراب 100 مل   28.5
2  4331               TOMEX PLUS 50 TAB               تومكس بلس 50 قرص   60.0
3  1022     TAROLIMUS 0.03% OINT. 15 GM     تاروليمس 0.03 % مرهم 15 جم  129.0
4   116  GLIPTUS PLUS 50/1000 MG 30 TAB  جليبتس بلس 50/1000 مجم 30 قرص  192.0

In [6]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


## 1. Data Preprocessing

### a. Cleaning

In [7]:
# remove punctuation and convert to lowercase
dataset_df['seller_item_name'] = dataset_df['seller_item_name'].map(lambda s: re.sub(r'[^\w\s]','',s.lower()))

#### Generating Dictionary from master file

In [8]:
master_df.duplicated().sum()

0

In [9]:
master_df.isna().sum()

sku                0
product_name       0
product_name_ar    0
price              0
dtype: int64

In [10]:
def normalize_english(text):
    text = str(text).lower()
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # remove non-alphanumeric characters
    # text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip()
    return text


def normalize_arabic(text):
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove Arabic diacritics (tashkeel)
    # text = re.sub(r'[^\u0621-\u064A\s]', '', text) # remove non-arabic characters
    text = re.sub(r'[^\u0621-\u064A0-9\s]', ' ', text) # remove non-arabic characters except for numbers
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = str(text).strip()
    return text

In [11]:
normalize_english(" we.ork//  / Hello,World!  ")

'we ork hello world'

In [12]:
normalize_arabic("helloo فيدروب 2800 وحدة دولية//مل نقط بالفم helloo 15 مل")

'فيدروب وحدة دولية مل نقط بالفم مل'

In [13]:
master_df['product_name_clean'] = master_df['product_name'].map(normalize_english)
master_df['product_name_ar_clean'] = master_df['product_name_ar'].map(normalize_arabic)

In [14]:
master_df.head()

sku                    product_name                product_name_ar  price  \
0   279          ANAFRONIL 75 MG 20 TAB  انافرونيل 75 مجم اس ار 20 قرص   75.0   
1  2282         LOPRECOUGH SYRUP 100 ML           لوبريكاف شراب 100 مل   28.5   
2  4331               TOMEX PLUS 50 TAB               تومكس بلس 50 قرص   60.0   
3  1022     TAROLIMUS 0.03% OINT. 15 GM     تاروليمس 0.03 % مرهم 15 جم  129.0   
4   116  GLIPTUS PLUS 50/1000 MG 30 TAB  جليبتس بلس 50/1000 مجم 30 قرص  192.0   

    product_name_clean    product_name_ar_clean  
0     anafronil mg tab  انافرونيل مجم اس ار قرص  
1  loprecough syrup ml         لوبريكاف شراب مل  
2       tomex plus tab            تومكس بلس قرص  
3    tarolimus oint gm         تاروليمس مرهم جم  
4  gliptus plus mg tab       جليبتس بلس مجم قرص

In [62]:
from collections import Counter

english_words = Counter(" ".join(master_df['product_name_clean']).split())
arabic_words = Counter(" ".join(master_df['product_name_ar_clean']).split())

# filter out one-letter words
english_words = {k:v for k,v in english_words.items() if len(k) > 2}
arabic_words = {k:v for k,v in arabic_words.items() if len(k) > 2}

english_words = Counter(english_words)
arabic_words = Counter(arabic_words)

In [63]:
english_words

Counter({'tab': 509,
         'cap': 129,
         'syrup': 44,
         'plus': 38,
         'amp': 37,
         'eye': 37,
         'cream': 36,
         'drops': 35,
         'susp': 27,
         'sachets': 18,
         'mcg': 16,
         'milk': 16,
         'spray': 15,
         'nasal': 14,
         'drop': 13,
         'vial': 12,
         'oint': 10,
         'oral': 10,
         'dose': 10,
         'erastapex': 9,
         'baby': 9,
         'solution': 8,
         'gel': 8,
         'adult': 8,
         'with': 8,
         'advance': 8,
         'limitless': 8,
         'panadol': 6,
         'vaginal': 6,
         'empacoza': 6,
         'lozenges': 6,
         'syringe': 6,
         'hero': 6,
         'tareg': 6,
         'pre': 6,
         'filled': 6,
         'voltaren': 6,
         'supp': 5,
         'extra': 5,
         'conventin': 5,
         'sachet': 5,
         'amaryl': 5,
         'tritace': 5,
         'retard': 5,
         'dolcyl': 5,
         'vit': 5,


In [64]:
def save_symspell_dict(words_dict, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for word, freq in words_dict.items():
            f.write(f"{word} 1000000\n")

In [65]:
save_symspell_dict(english_words, "symspell_english.txt")
save_symspell_dict(arabic_words, "symspell_arabic.txt")

In [99]:
from symspellpy import SymSpell, Verbosity

sym_spell_en = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell_ar = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell_en.load_dictionary("symspell_english.txt", term_index=0, count_index=1)
sym_spell_ar.load_dictionary("symspell_arabic.txt", term_index=0, count_index=1, encoding="utf-8")

True

In [101]:
def correct_spelling(sentence):
    words = sentence.split()  # Split sentence into words
    corrected_words = []

    for word in words:
        suggestions = sym_spell_ar.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected_word = suggestions[0].term if suggestions else word  # Use first suggestion
        corrected_words.append(corrected_word)

    return " ".join(corrected_words)  # Recombine corrected words

In [102]:
dataset_df['seller_item_name_clean'] = dataset_df['seller_item_name'].map(correct_spelling)

In [103]:
correct_spelling("كسبول")

'كبسول'

In [104]:
correct_spelling("تيلفاستت")

'تلفاست'

In [105]:
correct_spelling("كلوبترا")

'كلوباترا'

In [106]:
correct_spelling("دانسيت")

'دانست'

In [107]:
dataset_df[dataset_df['seller_item_name'].str.contains("كبسول")]

sku  marketplace_product_name_ar                   seller_item_name  \
7      1322     استوهالت 40 مجم 14 كبسول  استوهالت 40 مجم 1 شريط  14 كبسولة   
9      1322     استوهالت 40 مجم 14 كبسول              استوهالت 40 كبسول س ج   
10     1322     استوهالت 40 مجم 14 كبسول                  استوهالت 40 كبسول   
11     1322     استوهالت 40 مجم 14 كبسول                  استوهالت 40 كبسول   
16     1322     استوهالت 40 مجم 14 كبسول        استوهالت  40مجم ــ 14كبسولة   
...     ...                          ...                                ...   
82217   712  نيوروباتكس 600 مجم 20 كبسول           نيوروباتكس س ج 20 كبسوله   
82248   712  نيوروباتكس 600 مجم 20 كبسول      نيوروباتكس 600مجم 20كبسول س ج   
82249   712  نيوروباتكس 600 مجم 20 كبسول      نيوروباتكس 600مجم 20كبسول س ج   
82250   712  نيوروباتكس 600 مجم 20 كبسول        نيوروباتكس 600مجم 20 كبسولة   
82463  8771       ار اكس كريم مساج 50 جم                  نيوروباتكس كبسول    

       price            seller_item_name_clean  
7       56.5  استوهالت 40 مجم 1 شريط 14 كبسولة  
9       56.5         استوهالت 40 كبسول بلس مجم  
10      56.5                 استوهالت 40 كبسول  
11      56.5                 استوهالت 40 كبسول  
16      56.5            استوهالت مجم ــ كبسولة  
...      ...                               ...  
82217  134.0       نيوروباتكس بلس مجم 20 كبسول  
82248  134.0   نيوروباتكس 600مجم كبسول بلس مجم  
82249  134.0   نيوروباتكس 600مجم كبسول بلس مجم  
82250  134.0       نيوروباتكس 600مجم 20 كبسولة  
82463   75.0                  نيوروباتكس كبسول  

[6293 rows x 5 columns]

In [108]:
correct_spelling("جابتين")

'جابتن'

In [114]:
correct_spelling("جدييييد")

'جدييييد'

In [109]:
dataset_df.sample(10)

sku       marketplace_product_name_ar  \
54564  2350            اتورستات 20 مجم 14 قرص   
40979  1905           جاروبرايد 50 مجم 30 قرص   
11336  1538            انتوبرال 20 مجم 14 قرص   
71329  1631             افيميو قطرة عين 10 مل   
44914   860     باوريكتا 10 مجم 6 قرص أستحلاب   
30194  3021                اميولانت 20 كبسولة   
15691  1817  اكس تينشن بلس 150/12.5مجم 28 قرص   
29549  2506          ريموواكس نقط للاذن 15 مل   
6373    319                اتور 20 مجم 10 قرص   
49333   334            جابيماش 300 مجم 30 قرص   

                             seller_item_name  price  \
54564                  اتورستات 20 مجم اقراص    64.0   
40979                     جاروبرايد 30 قرص سج   63.0   
11336        انتوبرال 20مجم 14 قرص سعر جديد     61.0   
71329                         افيميو قطره س ج   44.0   
44914        باوريكتا 10مج 6قللتفتت بالفمايفا   45.0   
30194                 اميولانت 20كبسول س جديد   30.0   
15691  اكستينشن 150 بلاس 4شريط جديييييييييييد   76.0   
29549                        ريموواكس نقط س ج   24.0   
6373               اتور20 مج اقراص  59 ج جديد   59.0   
49333           جابيماش 300مجم 30قرص سعر جديد   85.5   

                    seller_item_name_clean  
54564                اتورستات 20 مجم اقراص  
40979                جاروبرايد 30 قرص مساج  
11336         انتوبرال مجم 14 قرص كير جديد  
71329                  افيميو قطرة بلس مجم  
44914     باوريكتا 10مج 6قللتفتت بالفمايفا  
30194              اميولانت كبسول بلس جديد  
15691  اكتينون 150 بلس شريط جديييييييييييد  
29549                 ريموواكس نقط بلس مجم  
6373            اتور مجم اقراص 59 مجم جديد  
49333          جابيماش 300مجم قرص كير جديد

In [24]:
# search for @ in the seller_item_name
dataset_df[dataset_df['seller_item_name'].str.contains(".")]

sku  marketplace_product_name_ar            seller_item_name  price  \
0      1322     استوهالت 40 مجم 14 كبسول       estohalt 40 mg 14 cap   56.5   
1      1322     استوهالت 40 مجم 14 كبسول        استوهالت 40 مجم 14 ك   56.5   
2      1322     استوهالت 40 مجم 14 كبسول        استوهالت 40 مجم 14 ك   56.5   
3      1322     استوهالت 40 مجم 14 كبسول        استوهالت 40 مجم 14 ك   56.5   
4      1322     استوهالت 40 مجم 14 كبسول        استوهالت 40 مجم 14 ك   56.5   
...     ...                          ...                         ...    ...   
83557  1603  سبانيلا ام آر 35 مجم 30 قرص             سبانيلا 35 مم ق   39.0   
83558  1603  سبانيلا ام آر 35 مجم 30 قرص         سبانيلا 35 مل اقراص   39.0   
83559  1603  سبانيلا ام آر 35 مجم 30 قرص    سبانيلا 35 مجم مر 30 قرص   39.0   
83560  1603  سبانيلا ام آر 35 مجم 30 قرص  سبانيلا 35 مجم ام ار اقراص   39.0   
83561  1603  سبانيلا ام آر 35 مجم 30 قرص    سبانيلا 35 مجم اقراص س ج   39.0   

           seller_item_name_clean  
0           estohalt 40 mg 14 cap  
1            استوهالت 40 مجم 14 ك  
2            استوهالت 40 مجم 14 ك  
3            استوهالت 40 مجم 14 ك  
4            استوهالت 40 مجم 14 ك  
...                           ...  
83557             سبانيلا 35 مم ق  
83558         سبانيلا 35 مل اقراص  
83559    سبانيلا 35 مجم مر 30 قرص  
83560  سبانيلا 35 مجم ام ار اقراص  
83561    سبانيلا 35 مجم اقراص س ج  

[83562 rows x 5 columns]

In [25]:
dataset_df.head()

sku marketplace_product_name_ar       seller_item_name  price  \
0  1322    استوهالت 40 مجم 14 كبسول  estohalt 40 mg 14 cap   56.5   
1  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5   
2  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5   
3  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5   
4  1322    استوهالت 40 مجم 14 كبسول   استوهالت 40 مجم 14 ك   56.5   

  seller_item_name_clean  
0  estohalt 40 mg 14 cap  
1   استوهالت 40 مجم 14 ك  
2   استوهالت 40 مجم 14 ك  
3   استوهالت 40 مجم 14 ك  
4   استوهالت 40 مجم 14 ك

In [26]:
X = dataset_df['seller_item_name']
y = dataset_df['sku']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [28]:
len(X_train), len(X_test)

(66849, 16713)